In [1]:
import requests 
import read_write_qrcd as q_reader

In [95]:
def del_solr_data(solr_core='qa_core', url='http://localhost:8983/solr/'):
    import requests
    headers = { 'Content-Type': 'text/xml'}
    params = { 'commit': 'true'}
    data = '<delete><query>*:*</query></delete>'

    response = requests.post(
        '{}{}/update'.format(url, solr_core), 
        headers=headers, 
        params=params, 
        data=data)
    return response

In [96]:
r = del_solr_data()
r.text

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">206</int>\n</lst>\n</response>\n'

In [67]:
def list2str(data_list):
    result = '[\n'
    for d in data_list:
        # print(d)
        doc_id = d.get('id')
        doc = d.get('doc')
        result+= '{id:' + doc_id + 'doc:' + doc +  '},\n'
    result+= ']\n'
    return result 

In [74]:
def post_data_list_solr(solr_core='qa_core',  url='http://localhost:8983/solr/', data=''):
    import requests
    headers = {'Content-Type': 'application/json'}
    response = requests.post(
        '{}{}/update'.format(url, solr_core), 
        headers=headers, 
        data=data)
    return response

In [99]:
def post_data_solr(solr_core='qa_core',  url='http://localhost:8983/solr/', data_dict={}):
    import requests
    import json
    params = { 'commit': 'true'}
    response = requests.post(
        url='{}{}/update/json/docs'.format(url, solr_core), 
        params=params,
        json=json.dumps(data_dict))
    return response

In [2]:
train_data_file = '../quranqa/datasets/qrcd_v1.1_train.jsonl'
train = q_reader.load_jsonl(train_data_file)

Loaded 710 records from ../quranqa/datasets/qrcd_v1.1_train.jsonl


In [23]:
def prepare_data(data_item):
    result_list = []
    
    pq_id = data_item['pq_id'].split(':')[0]
    verse_start = int(data_item['verses'].split('-')[0])
    passages = [ver.strip() for ver in data_item.get('passage').split('.') if ver.strip()]
    for passage in passages:
        doc = {
            'id': pq_id + '_' + str(verse_start),
            'doc': passage
        }
        verse_start+=1
        result_list.append(doc)

    return result_list

In [24]:
i0 = train[0]
print(i0)

{'pq_id': '2:8-16_364', 'passage': 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين. يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون. في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون. وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون. ألا إنهم هم المفسدون ولكن لا يشعرون. وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون. وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون. الله يستهزئ بهم ويمدهم في طغيانهم يعمهون. أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين.', 'surah': 2, 'verses': '8-16', 'question': 'لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ""من يضلل الله فما له من هاد"" كما ورد من قوله تعالى في آية 23 و آية 36 من سورة الزمر؟', 'answers': [{'text': 'أولئك الذين اشتروا الضلالة بالهدى', 'start_char': 504}]}


In [65]:
docs_data = prepare_data(i0)
docs_data

[{'id': '2_8',
  'doc': 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين'},
 {'id': '2_9',
  'doc': 'يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون'},
 {'id': '2_10',
  'doc': 'في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون'},
 {'id': '2_11',
  'doc': 'وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون'},
 {'id': '2_12', 'doc': 'ألا إنهم هم المفسدون ولكن لا يشعرون'},
 {'id': '2_13',
  'doc': 'وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون'},
 {'id': '2_14',
  'doc': 'وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون'},
 {'id': '2_15', 'doc': 'الله يستهزئ بهم ويمدهم في طغيانهم يعمهون'},
 {'id': '2_16',
  'doc': 'أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين'}]

In [100]:
d = docs_data[5]
print(d)
r = post_data_solr(data_dict=d)
r.text

{'id': '2_13', 'doc': 'وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون'}


'{\n  "responseHeader":{\n    "status":0,\n    "QTime":9}}\n'

In [72]:
data_str = list2str(docs_data)

In [33]:
docs_data[0]

{'id': '2_8',
 'doc': 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين'}

In [111]:
del_solr_data()

<Response [200]>

In [108]:
import pysolr
solr = pysolr.Solr(
    'http://localhost:8983/solr/qa_core', 
    always_commit=True)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":0,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"-35"}},\n  "status":"OK"}\n'

In [109]:
docs_data

[{'id': '2_8',
  'doc': 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين'},
 {'id': '2_9',
  'doc': 'يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون'},
 {'id': '2_10',
  'doc': 'في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون'},
 {'id': '2_11',
  'doc': 'وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون'},
 {'id': '2_12', 'doc': 'ألا إنهم هم المفسدون ولكن لا يشعرون'},
 {'id': '2_13',
  'doc': 'وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون'},
 {'id': '2_14',
  'doc': 'وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون'},
 {'id': '2_15', 'doc': 'الله يستهزئ بهم ويمدهم في طغيانهم يعمهون'},
 {'id': '2_16',
  'doc': 'أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين'}]

In [112]:
solr.add(docs_data)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":216}}\n'

In [116]:
results = solr.search(q='*:*', fl='id,doc,score', df='doc')
for r in results:
    print("id: {}. doc: {}. score: {}".format(r['id'], r['doc'], r['score']))

id: 2_8. doc: ['ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين']. score: 1.0
id: 2_9. doc: ['يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون']. score: 1.0
id: 2_10. doc: ['في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون']. score: 1.0
id: 2_11. doc: ['وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون']. score: 1.0
id: 2_12. doc: ['ألا إنهم هم المفسدون ولكن لا يشعرون']. score: 1.0
id: 2_13. doc: ['وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون']. score: 1.0
id: 2_14. doc: ['وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون']. score: 1.0
id: 2_15. doc: ['الله يستهزئ بهم ويمدهم في طغيانهم يعمهون']. score: 1.0
id: 2_16. doc: ['أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين']. score: 1.0


In [117]:
solr.delete(q='*:*')

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">199</int>\n</lst>\n</response>\n'

In [118]:
results = solr.search(q='*:*', fl='id,doc,score', df='doc')
for r in results:
    print("id: {}. doc: {}. score: {}".format(r['id'], r['doc'], r['score']))